<center><h1>Projet IA - Fake News & Deep Fake</h1></center>

<center><h1>Présenté par Shabboo ALEAGHA & Ophélie ENGASSER</h1></center>

<center><h2>M2-IADS-EL - Mai 2024</h2></center>

# Scraping de données

Ce notebook a pour objectif de récupérer des articles issus de journaux en ligne (sections 'politique'), afin de pouvoir les utiliser comme données non vues par les modèles pour tester leurs performances "en production". Pour récupérer ces articles nous utiliserons la méthode du webscraping via la librairie Python BeautifulSoup. Nous allons récupérer des données Real et Fake, dans 2 journaux différents:
- Les données Fake seront scrapées sur le site du journal satirique américain "The Onion" (journal ayant inspiré notre journal satirique français "Le Gorafi").
- Les données Real seront scrapées sur le site du journal américain "Time Magazine". 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import time, sleep
from random import randint
from IPython.display import clear_output
from warnings import warn
warn('Attention')

C:\Users\Engasser Ophélie\AppData\Local\Temp\ipykernel_11204\607999700.py:8: UserWarning: Attention
  warn('Attention')


## Scraping de données Fake: articles issus du journal satirique américain "The Onion"

In [2]:
# boucle pour récupérer les liens des articles

pages = [str(i) for i in range(0, 2301, 20)]
links = []
start_time = time()
page_count = 0
req = 0

for page in pages:
    url = 'https://www.theonion.com/politics?startIndex=' + page
    response = requests.get(url, allow_redirects=False)
    req += 1
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
    clear_output(wait=True)

    if response.status_code == 200:
        page_count += 1
        soup = BeautifulSoup(response.content, 'html.parser')
        containers = soup.find('div', class_='sc-17uq8ex-0 fakHlO')

        if containers:
            cards = containers.find_all('div', class_='sc-cw4lnv-13 hHSpAQ')
            for card in cards:
                a_tags = card.find_all('a', class_='sc-1out364-0 dPMosf js_link', href=True)
                if a_tags:
                    link = a_tags[0]['href']
                    links.append(link)
                else:
                    pass
        else:
            print("Aucun conteneur trouvé sur la page :", url)
    else:
        print("La requête a échoué pour la page :", url)

Request: 116; Frequency: 1.2838434701813222 requests/s


In [4]:
len(cards)

20

In [5]:
len(containers)

20

In [6]:
links

['https://www.theonion.com/trump-kisses-supporter-s-burger-1851339626',
 'https://www.theonion.com/what-trump-meant-by-there-would-be-a-bloodbath-if-he-1851346115',
 'https://www.theonion.com/everything-lara-trump-plans-to-do-after-rnc-takeover-1851339709',
 'https://www.theonion.com/trump-dehumanizes-migrants-by-referring-to-them-as-dons-1851345138',
 'https://www.theonion.com/kamala-harris-discovers-classified-document-revealing-s-1851339713',
 'https://www.theonion.com/vaseline-covered-trump-reverses-tiktok-stance-after-get-1851325670',
 'https://www.theonion.com/i-know-him-kamala-harris-whispers-to-mike-johnson-du-1851316776',
 'https://www.theonion.com/biden-crumbles-to-dust-during-state-of-union-1851313715',
 'https://www.theonion.com/pros-and-cons-of-voting-uncommitted-in-the-democratic-1851312889',
 'https://www.theonion.com/marianne-williamson-successfully-primaries-biden-in-all-1851298077',
 'https://www.theonion.com/trump-scared-to-check-credit-score-1851297917',
 'https://w

In [7]:
len(links)

2312

In [8]:
# récupération des articles à partir des liens stockés dans links

liens = [str(i) for i in links] # pour automatiser les URL dans la boucle
page_line_count = len(liens) # pour baliser le nb de requêtes
articles = []
start_time = time()
req = 0

for lien in liens:
    url = lien
    response = requests.get(url)
    req += 1
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
    clear_output(wait = True)
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(req, response.status_code))
    if req > page_line_count:
        warn('Nombre de requêtes trop important')
        break
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.body
    container = body.find_all('p', class_='sc-77igqf-0 fnnahv')
    article_text = ' '.join([c.get_text(strip=True) for c in container])

    if article_text.strip():
        articles.append(article_text)
    else:
        print("Article vide pour l'URL :", url)

Request: 2312; Frequency: 1.3818992651614999 requests/s


In [9]:
len(articles)

1710

In [10]:
# dataframe
df_theonion = pd.DataFrame({
    'articles': articles
})

In [11]:
df_theonion['articles'][1]

'Presidential candidate Donald Trump recently remarked that if he loses the 2024 election it will “be a bloodbath for the country.”The Onionexamines what he might have meant by the statement. Pretty straightforward, honestly. Stabbing and stabbing and stabbing with a sharp, blood-stained knife can often get messy. At least he came right out and said it. Awwwww! Look at that little droopy face! Look at it! No need to panic. Trump doesn’t plan on committing violence against anyone other than his loved ones. When Trump hosted the popular reality television program, he famously referenced the bloodshed of his enemies and called immigrants animals. What you interpreted when you heard him predict a bloodbath says more about you than about Trump. He was merely holding up a mirror to your subconscious. The rally caught him in a moment of panic after using the restroom. Although the ‘bloodbath’ comments sound alarming out of context, Trump was simply threatening to personally lock the doors of 

In [12]:
# exportation en csv
df_theonion.to_csv('data_new_fake_theonion.csv', index=True)

# Scraping de données True : articles issus du Time Magazine

In [2]:
url = "https://time.com/section/politics/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [36]:
containers = soup.find('div', class_='component taxonomy-related-touts section-related__touts')

In [41]:
links = []

touts = soup.find_all('div', class_='taxonomy-tout')

for tout in touts:
    link = tout.find('a')
    if link:
        href = link.get('href')
        links.append(href)

['/6965490/senate-mayorkas-impeachment-trial/', '/6966558/donald-trump-hush-money-trial-politics-election/', '/6966758/how-the-u-s-rallied-to-defend-israel-from-irans-massive-attack/', '/6966719/joe-biden-national-security-israel-iran-support/', '/6966465/donald-trump-universities-agenda-antisemitism/', '/6966390/kamala-harris-blame-donald-trump-abortion-arizona/', '/6966314/biden-assange-wikileaks-extradition/', '/6965602/arizona-abortion-ban-politics-2024/', '/6965419/trump-hush-money-trial-legal-questions/', '/6965382/biden-netanyahu-war-comments/', '/6965211/donald-trump-abortion-religious-conservatives-politics/', '/6965160/trump-nationwide-abortion-ban-lies-essay/', '/6964826/biden-new-student-loan-forgiveness-plan/', '/6964678/donald-trump-abortion-ban-states/', '/6964364/exclusive-no-tech-for-apartheid-google-workers-protest-project-nimbus-1-2-billion-contract-with-israel/', '/6964462/bernie-sanders-office-fire-man-arrested-charged-motive/', '/6964255/trump-judges-daughter-gag-

In [46]:
# boucle pour récupérer les liens des articles

pages = [str(i) for i in range(0, 101)]
links = []
start_time = time()
page_count = 0
req = 0

for page in pages:
    url = 'https://time.com/section/politics/?page=' + page
    response = requests.get(url, allow_redirects=False)
    req += 1
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
    clear_output(wait=True)

    if response.status_code == 200:
        page_count += 1
        soup = BeautifulSoup(response.content, 'html.parser')
        containers = soup.find('div', class_='component taxonomy-related-touts section-related__touts')

        touts = soup.find_all('div', class_='taxonomy-tout')

        for tout in touts:
            link = tout.find('a')
            if link:
                href = link.get('href')
                links.append(href)

Request: 101; Frequency: 0.6614368951951668 requests/s


In [47]:
links

['/6967119/joe-biden-israel-iran-national-security-campaign/',
 '/6965490/senate-mayorkas-impeachment-trial/',
 '/6966558/donald-trump-hush-money-trial-politics-election/',
 '/6966758/how-the-u-s-rallied-to-defend-israel-from-irans-massive-attack/',
 '/6966719/joe-biden-national-security-israel-iran-support/',
 '/6966465/donald-trump-universities-agenda-antisemitism/',
 '/6966390/kamala-harris-blame-donald-trump-abortion-arizona/',
 '/6966314/biden-assange-wikileaks-extradition/',
 '/6965602/arizona-abortion-ban-politics-2024/',
 '/6965419/trump-hush-money-trial-legal-questions/',
 '/6965382/biden-netanyahu-war-comments/',
 '/6965211/donald-trump-abortion-religious-conservatives-politics/',
 '/6965160/trump-nationwide-abortion-ban-lies-essay/',
 '/6964826/biden-new-student-loan-forgiveness-plan/',
 '/6964678/donald-trump-abortion-ban-states/',
 '/6964364/exclusive-no-tech-for-apartheid-google-workers-protest-project-nimbus-1-2-billion-contract-with-israel/',
 '/6964462/bernie-sanders-o

In [48]:
len(links)

2000

In [50]:
# récupération des articles à partir des liens stockés dans links

liens = [str(i) for i in links] # pour automatiser les URL dans la boucle
page_line_count = len(liens) # pour baliser le nb de requêtes
articles = []
start_time = time()
req = 0

for lien in liens:
    url = 'https://time.com' + lien
    response = requests.get(url)
    req += 1
    elapsed_time = time() - start_time
    print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
    clear_output(wait = True)
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(req, response.status_code))
    if req > page_line_count:
        warn('Nombre de requêtes trop important')
        break
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.body
    container = body.find_all('p', class_='self-baseline px-0 font-pt-serif text-17px leading-7 tracking-0.5px')
    article_text = ' '.join([c.get_text(strip=True) for c in container])

    if article_text.strip():
        articles.append(article_text)
    else:
        print("Article vide pour l'URL :", url)

Request: 2000; Frequency: 1.0345237455889822 requests/s


In [51]:
len(articles)

1981

In [53]:
articles[1]

'House Republicans are preparing to sign articles of impeachment against Department of Homeland Security Secretary Alejandro Mayorkas and walk them across the Capitol Rotunda to the Senate as soon as this week. That will make Mayorkas the first cabinet official to beformally impeached in 150 years. But what happens next is far from certain. Only one thing is for sure: it\'s going to get messy. Democrats control the Senate and almost certainly will vote to acquit Mayorkas, arguing that the articles of impeachment against him are deeply flawed and based on Republican objections to the Biden Administration’s immigration policy—not on any alleged high crimes and misdemeanors. The question is, how long will the process drag out? Both parties have procedural tools at their disposal. The top Democrat in the Senate, Majority Leader Chuck Schumer,toldreporters April 9that he thought he had the votes to rapidly close up the impeachment proceedings in the Senate. “We’re going to try and resolve t

In [55]:
# dataframe
df_time = pd.DataFrame({
    'articles': articles
})

In [56]:
# exportation en csv
df_time.to_csv('data_new_real_time.csv', index=True)